In [1]:
!pip install tensorflow
!pip install scikit-learn
!pip install Pillow  # For image loading and preprocessing (used by `load_img` and `img_to_array`)
import sys
sys.path.append("Desktop/MachineLearning/ml hackathon/66e31d6ee96cd_student_resource_3/student_resource 3/src")  # Adjust to the correct path
!pip install tensorflow scikit-learn Pillow
!pip install tqdm




ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [ ]:
import pandas as pd
import numpy as np
import os
from src import constants
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from src.utils import download_images  # Ensure this module works for your environment
from src.constants import allowed_units

# Load and preprocess data
train_data = pd.read_csv('dataset/train.csv')
test_data = pd.read_csv('dataset/test.csv')

# Download train images
# Define the folder where you want to download the images
download_folder = r"C:\Users\91852\AppData\Local\Programs\Python\Python312\ml hackathon\66e31d6ee96cd_student_resource_3\student_resource 3"

# Download train images
train_data['image_path'] = train_data['image_link'].apply(lambda link: download_images(link, download_folder))


# Preprocess images
def load_images(image_paths, target_size=(224, 224)):
    images = []
    for img_path in image_paths:
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img)
        img = tf.keras.applications.efficientnet.preprocess_input(img)
        images.append(img)
    return np.array(images)

from multiprocessing.pool import ThreadPool

pool = ThreadPool(processes=60)  # Set below the 63 handle limit

from concurrent.futures import ThreadPoolExecutor

def download_images_parallel(links, download_folder):
    with ThreadPoolExecutor(max_workers=4) as executor:  # Adjust the number of workers
        executor.map(lambda link: download_images(link, download_folder), links)

def process_in_batches(links, batch_size, download_folder):
    for i in range(0, len(links), batch_size):
        batch = links[i:i + batch_size]
        download_images_parallel(batch, download_folder)

process_in_batches(train_data['image_link'].tolist(), batch_size=50, download_folder=download_folder)

X = load_images(train_data['image_path'].values)

# Extract the entity_value and convert it to a float for training
y = train_data['entity_value'].apply(lambda val: float(val.split()[0])).values  # Extract numerical part
unit_labels = train_data['entity_value'].apply(lambda val: val.split()[1])  # Extract units

# Encode unit labels if needed (you can use a separate model for unit prediction)
unit_mapping = {unit: i for i, unit in enumerate(ALLOWED_UNITS)}
y_units = unit_labels.map(unit_mapping).values

# Split the data into training and validation sets
X_train, X_val, y_train, y_val, y_units_train, y_units_val = train_test_split(X, y, y_units, test_size=0.2, random_state=42)

# Model architecture
def build_model(input_shape):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False  # Freeze pre-trained layers

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='linear')  # Predict numerical entity value
    ])
    return model

# Compile model
input_shape = (224, 224, 3)
model = build_model(input_shape)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# Load test images for prediction
test_data['image_path'] = test_data['image_link'].apply(lambda link: download_images(link))
X_test = load_images(test_data['image_path'].values)

# Predict on test set
y_pred = model.predict(X_test)

# Placeholder for unit prediction (simple mapping here, can be replaced with an advanced model)
y_units_pred = np.random.choice(list(unit_mapping.keys()), len(y_pred))  # Dummy predictions for now

# Format output
predictions = []
for i, pred in enumerate(y_pred):
    pred_value = f"{float(pred):.2f} {y_units_pred[i]}"
    predictions.append(pred_value)

output = pd.DataFrame({
    'index': test_data['index'],
    'prediction': predictions
})

# Save predictions to a CSV file
output.to_csv('predictions.csv', index=False)

# Run sanity checker
os.system('python src/sanity.py predictions.csv')
